In [5]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import lifetimes 
#import tune_sklearn
#import hyperopt


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_validate
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from pipelinehelper import PipelineHelper
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, StackingClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB, CategoricalNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, f1_score, accuracy_score, precision_score, confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve
from tune_sklearn import TuneSearchCV
from ray import tune
#from hyperopt import hp
from collections import Counter
from imblearn.under_sampling import NearMiss, RandomUnderSampler, AllKNN, ClusterCentroids, CondensedNearestNeighbour, EditedNearestNeighbours, InstanceHardnessThreshold, NeighbourhoodCleaningRule, OneSidedSelection, RepeatedEditedNearestNeighbours, TomekLinks
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler, SVMSMOTE, SMOTEN, SMOTENC, BorderlineSMOTE, KMeansSMOTE
from sklearn.svm import SVC, LinearSVC, OneClassSVM
from imblearn.ensemble import BalancedRandomForestClassifier, BalancedBaggingClassifier, EasyEnsembleClassifier, RUSBoostClassifier

import os

import tqdm
from tqdm import tqdm
from sklearn.metrics import make_scorer

In [6]:
train_old_data = pd.read_csv(r"C:\Users\Roma\Hack #1\Processed data\df_train_anyclass.csv", index_col=[0])
test_old_data = pd.read_csv(r"C:\Users\Roma\Hack #1\Processed data\df_test_anyclass.csv", index_col=[0])

In [7]:
scaler_old_data = StandardScaler()

scal_train_old = scaler_old_data.fit_transform(train_old_data.drop('blocked', axis=1))
scal_test_old = scaler_old_data.transform(test_old_data)

scal_train_old = pd.DataFrame(scal_train_old, columns=train_old_data.drop('blocked', axis=1).columns)
scal_train_old.set_index(train_old_data.index, inplace=True)
scal_train_old['blocked'] = train_old_data['blocked']

scal_test_old = pd.DataFrame(scal_test_old, columns=test_old_data.columns)
scal_test_old.set_index(test_old_data.index, inplace=True)

In [11]:
bernouli_main_test = pd.read_csv(r"C:\Users\Roma\Desktop\TextDateSVMSMOTE_BernoulliNB (1).csv", index_col=[0])

In [19]:
X_old = scal_train_old.drop('blocked', axis = 1)
Y_old = scal_train_old['blocked']

In [24]:
nm = SVMSMOTE(sampling_strategy='auto', k_neighbors=14, m_neighbors=10, out_step=1)
X_res_old, y_res_old = nm.fit_resample(X_old, Y_old)
X_train_res_old, X_test_res_old, y_train_res_old, y_test_res_old = train_test_split(X_res_old, y_res_old, test_size=0.35, stratify=y_res_old)


gnb_clf = BernoulliNB(binarize=4.1)
gnb_clf.fit(X_train_res_old, y_train_res_old)

print(recall_score(y_test_res_old, gnb_clf.predict(X_test_res_old), average='macro'))
print(f1_score(bernouli_main_test['blocked'], gnb_clf.predict(scal_test_old), average='macro'))

0.8132757690232055
0.9605978426102714


In [ ]:
gnb_clf.fit(X_res_old, y_res_old)
print(f1_score(bernouli_main_test['blocked'], gnb_clf.predict(scal_test_old), average='macro'))

In [ ]:
pred_gnb = gnb_clf.predict(scal_test_old)
pred_gnb = pd.DataFrame(pred_gnb)

pred_gnb['contract_id'] = scal_test_old.index
pred_gnb = pred_gnb.reindex(columns = ['contract_id', 0])
pred_gnb.rename(columns={0:'blocked'}, inplace=True)

In [ ]:
pred_gnb['blocked'].value_counts()

In [ ]:
print(f1_score(bernouli_main_test['blocked'], gnb_clf.predict(scal_test_old), average='macro'))

In [ ]:
pred_gnb.to_csv(r'C:\Users\Roma\Hack #1\Processed data\TextDateSVMSMOTE_BernoulliNB(alpha=1, binarize = 4.1,  fit_prior = True)_old_simpleimputer_X_res_old_y_res_old.csv', columns=['contract_id', 'blocked'], index= False)
#Result: 79+